In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/My Drive"

os.chdir(path)
os.listdir(path)

In [ ]:
!pip install "tensorflow-gpu>=1.15.2,<2.0"

In [ ]:
!pip install ampligraph

In [ ]:
import ampligraph
ampligraph.__version__

In [7]:
#导包
import numpy as np
from ampligraph.datasets import load_wn18
from ampligraph.latent_features import TransE
from ampligraph.evaluation import evaluate_performance, mrr_score, hits_at_n_score
 
def main():
 
    #加载Wordnet18数据集
    X = load_wn18()
 
    # 用pairwise损失函数初始化TransE模型
    model = TransE(batches_count=10, seed=0, epochs=1200, k=150, eta=10,
                    # 使用adam优化器，学习率为1e-3
                    optimizer='adam', optimizer_params={'lr':1e-3},
                    #使用pairwise损失函数，margin值为0.5
                    loss='pairwise', loss_params={'margin':0.5},
                    #L2正则，正则权重为1e-5
                    regularizer='LP', regularizer_params={'p':2, 'lambda':1e-5}, 
                    # 显示进度条
                    verbose=True)
 
    #定义filter用于过滤替换头实体或尾实体生成负例时产生的正例样本
    filter = np.concatenate((X['train'], X['valid'], X['test']))
    
    #通过训练集、验证集训练模型
    model.fit(X['train'], 
              early_stopping = False,
              )
 
    #在测试集上进行评估
    #可以通过filter_triples=None不进行过滤
    ranks = evaluate_performance(X['test'], 
                                 model=model, 
                                 filter_triples=filter,
                                 use_default_protocol=True,#分别替换头实体、尾实体
                                 verbose=True)
 
    # 计算并打印评价指标:
    mrr = mrr_score(ranks)
    hits_10 = hits_at_n_score(ranks, n=10)
    print("MRR: %f, Hits@10: %f" % (mrr, hits_10))
 
if __name__ == "__main__":
    main()

Average TransE Loss:   0.000490: 100%|██████████| 1200/1200 [04:13<00:00,  4.74epoch/s]

WARNING - DeprecationWarning: use_default_protocol will be removed in future. Please use corrupt_side argument instead.



100%|██████████| 5000/5000 [00:33<00:00, 150.95it/s]


MRR: 0.402440, Hits@10: 0.824100
